In [22]:
import importlib
import subprocess

import Advanced_SQL as asql
import duckdb
import SQL as sql

_ = importlib.reload(asql)
_ = importlib.reload(sql)

Use this cell to test any sql queries. This can be done in one of the following ways:

1. Fill in the `qry` within the """ """.
2. Call the question functions that you have already filled in from the SQL.py and Advanced_SQL.py files. An example of question_1 being called from the SQL.py file is currently commented out.

This cell can be copied if you wish to run multiple queries.


In [110]:
qry = """alter table corrected_customers 
              add column AgeCategory varchar(20);


              update corrected_Customers
              set AgeCategory = case when CorrectedAge <20 then 'teen' 
                                     when CorrectedAge between 20 and 29 then 'Young Adult'
                                     when CorrectedAge between 30 and 59 then 'Adult'
                                     else 'Pensioner'
                                     end ;
                                     
              alter table corrected_customers
              add column Rank int ;

              with RepaymentTotals as ( select c.CustomerID, c.Age, c.CorrectedAge, c.Gender,c.AgeCategory,
              coalesce (count(r.RepaymentID),0) as TotalRepayments 
              from corrected_customers c 
              left join repayments r on c.CustomerID = r.CustomerID
              group by c.CustomerID, c.Age, c.Gender, c.CorrectedAge, c.Gender, c.AgeCategory),RankedCustomers 
              as (select CustomerID, Age, CorrectedAge, Gender, AgeCategory, TotalRepayments,
              Dense_Rank() over (partition by AgeCategory order by TotalRepayments desc ) as Rank
              from RepaymentTotals)

              update corrected_customers 
              set Rank = RankedCustomers.Rank 
              from RankedCustomers where corrected_customers.CustomerID = RankedCustomers.CustomerID;

              select CustomerID,Age,CorrectedAge, Gender,AgeCategory, Rank from corrected_customers;
              

              
              
              

                
"""

#qry = sql.question_1()


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerID,Age,CorrectedAge,Gender,AgeCategory,Rank
0,11,44,26,Male,Young Adult,9
1,19,43,49,Male,Adult,7
2,30,33,52,Male,Adult,9
3,35,80,48,Male,Adult,8
4,50,61,24,Male,Young Adult,8
...,...,...,...,...,...,...
995,953,48,38,Female,Adult,7
996,961,68,79,Female,Pensioner,8
997,978,44,34,Female,Adult,8
998,982,67,63,Female,Pensioner,5


Should you break/incorrectly update any database tables, the database can be reset by running the following cell


In [24]:
load_script_path = "database/database_load.py"
subprocess.run(["python3", load_script_path])

CompletedProcess(args=['python3', 'database/database_load.py'], returncode=0)